<a href="https://colab.research.google.com/github/imrahul1/Springboard/blob/master/Modelling_Black_Friday_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mention that we need to convert the feature variables to numerical types for preprocessing. Why?

Also do Unsupervised Learning.

And make printed notes of your learnings.


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#2.1 Get the file
downloaded = drive.CreateFile({'id':'11dlq7CXZahRY3Y2KRsh2qLnO8F8h9cT5'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')

<h3> Let's import the required packages first</h3>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

<h3>Data Wrangling</h3>

In [5]:
train = pd.read_csv('train.csv')
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


As we can see many values in the Product categories are NaN. Let's have a deeper look into the structure of the DataFrame.

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In the information we can see that product categories have lesser non-null entries than other columns.

In [7]:
train['Product_Category_1'].isnull().value_counts()

False    550068
Name: Product_Category_1, dtype: int64

Below we can see the Product_Category_2 column has many null observations.

In [8]:
train['Product_Category_2'].isnull().value_counts()

False    376430
True     173638
Name: Product_Category_2, dtype: int64

Here we can see that the non-null and null observations together make up the total entries

In [9]:
376430+173638

550068

In [10]:
train['Product_Category_3'].isnull().value_counts()

True     383247
False    166821
Name: Product_Category_3, dtype: int64

Even in the case of the column Product_Category_2 the null and non-null values together constitute the total number of observations.

In [11]:
383247+166821

550068

We can fill the null values as 0 because they were possibly not purchased during those transactions.

In [0]:
train.fillna(0, inplace=True)

In [13]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,6.735436,3.841941,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,6.215492,6.250712,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,0.000000,0.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,0.000000,0.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,5.000000,0.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,14.000000,8.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null float64
Product_Category_3            550068 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


<h2> Feature Engineering </h2>

In [0]:
train['Product_ID'] = train['Product_ID'].str.replace('P00', '')

In [17]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,069042,F,0-17,10,A,2,0,3,0.0,0.0,8370
1,1000001,248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,087842,F,0-17,10,A,2,0,12,0.0,0.0,1422
3,1000001,085442,F,0-17,10,A,2,0,12,14.0,0.0,1057
4,1000002,285442,M,55+,16,C,4+,0,8,0.0,0.0,7969


In [18]:
train['Product_ID'].dtype

dtype('O')

<h4> Encoding the features to make them fit for modelling.
<br>Initializing Label Encoder</h4>

In [0]:
le = LabelEncoder()

In [20]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,069042,F,0-17,10,A,2,0,3,0.0,0.0,8370
1,1000001,248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,087842,F,0-17,10,A,2,0,12,0.0,0.0,1422
3,1000001,085442,F,0-17,10,A,2,0,12,14.0,0.0,1057
4,1000002,285442,M,55+,16,C,4+,0,8,0.0,0.0,7969


In [0]:
cols = ['Product_ID', 'Gender', 'Occupation', 'Age', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 
        'Product_Category_2','Product_Category_3','Purchase']

In [0]:
for col in cols:
  train[col]=le.fit_transform(train[col])

In [23]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,672,0,0,10,0,2,0,2,0,0,6702
1,1000001,2376,0,0,10,0,2,0,0,5,11,12121
2,1000001,852,0,0,10,0,2,0,11,0,0,586
3,1000001,828,0,0,10,0,2,0,11,13,0,494
4,1000002,2734,1,6,16,2,4,0,7,0,0,6301


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null int64
Gender                        550068 non-null int64
Age                           550068 non-null int64
Occupation                    550068 non-null int64
City_Category                 550068 non-null int64
Stay_In_Current_City_Years    550068 non-null int64
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null int64
Product_Category_3            550068 non-null int64
Purchase                      550068 non-null int64
dtypes: int64(12)
memory usage: 50.4 MB


<h2>Scaling and Normalizing </h2>

In [0]:
train['User_ID']= le.fit_transform(train['User_ID'])

<h2>Scaling</h2>

In [0]:
num_col = ['Age', 'Occupation', 'Stay_In_Current_City_Years', 'Product_Category_1', 
           'Product_Category_2', 'Product_Category_3']

In [0]:
scale = StandardScaler()

In [0]:
for col in num_col:
    train[col] = scale.fit_transform(train[col].values.reshape(-1, 1))

<h2> Modelling</h2>

In [0]:
X = train.drop(['Purchase'], axis=1)
y = train['Purchase']

<h4>Calling the train test split</h4>

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.30, random_state=0, shuffle=True)

<h4>Initializing Linear Regression </h4>

In [0]:
linr=LinearRegression()

In [32]:
linr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = linr.predict(X_val)

In [34]:
linr.score(X_val,y_val)

0.1538312120753047

<h4>Scoring the predictions based on RMSE</h4>

In [35]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = print(np.sqrt(mean_squared_error(y_pred, y_val)))

3775.7493615969393


<h4>Importing and initializing Decision Tree Regressor</h4>

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

In [37]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [0]:
y_pred_rd=dtr.predict(X_val)

<h4>Scoring Decision Tree</h4>

In [39]:
dtr.score(X_val,y_val)

0.4298806750517838

In [40]:
print(np.sqrt(mean_squared_error(y_val, y_pred_rd)))

3099.2566225019336


<h4>Random Forest Regressor </h4>

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
rfr = RandomForestRegressor()

In [43]:
rfr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
y_pred_rfr = rfr.predict(X_val)

In [45]:
rfr.score(X_val, y_val)

0.6774292173766987

In [46]:
print(np.sqrt(mean_squared_error(y_val, y_pred_rfr)))

2331.24099030193


<h2>Cross Validation</h2>

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
cv_results = cross_val_score(rfr, X, y, cv=5)

In [49]:
print(cv_results)

[0.66920625 0.66959225 0.67003054 0.66702346 0.69556056]


In [50]:
np.mean(cv_results)

0.674282612534217

In [51]:
rfr.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

<h4>Default number of estimators in Random Forest Regressor is 10 and maximum depth is None</h4>

In [52]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'max_depth': [3, None],
     'n_estimators': (3,5,10)}
model = GridSearchCV(rfr, param_grid, cv=5)
model.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [3, None], 'n_estimato

In [53]:
model.best_params_

{'max_depth': None, 'n_estimators': 10}